### Merge Data

#### Merge Pediatrics data with Drug and Reaction
- Pediatrics data from 3_Pediatrics_data_merge.ipynb file
- Raw Drug file from drugcharacteristics.csv.gz (Drug)
- Reaction file from reactions.csv.gz (Meddra reaction PT)

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

#### Raw Data

In [12]:
ped_report_data = pd.read_csv('../../data/pediatric_patients_report_serious_reporter.csv.gz',compression='gzip', index_col=0)

/tmp/ipykernel_19135/852322303.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  ped_report_data = pd.read_csv('../../data/pediatric_patients_report_serious_reporter.csv.gz',compression='gzip', index_col=0)


In [13]:
drug_characteristics_df = pd.read_csv('../../data/openFDA_drug_event/er_tables_memory_efficient/drugcharacteristics.csv.gz',compression='gzip',index_col=0)

/tmp/ipykernel_19135/1365005143.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  drug_characteristics_df = pd.read_csv('../../data/openFDA_drug_event/er_tables_memory_efficient/drugcharacteristics.csv.gz',compression='gzip',index_col=0)


In [14]:
reaction_df = pd.read_csv("../../data/openFDA_drug_event/er_tables_memory_efficient/reactions.csv.gz", compression='gzip', low_memory=False, index_col=0)

##### Merge Data

In [15]:
ped_data = ped_report_data.copy()

In [16]:
# ped_report_data.isna().sum()
ped_report_data.shape

(859343, 21)

In [17]:
ped_data_merge = ped_report_data\
    .merge(drug_characteristics_df, on='safetyreportid', how='inner')\
    .merge(reaction_df, on='safetyreportid', how='inner')

##### Pediatrics Data Merge

In [18]:
ped_data_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1754671 entries, 0 to 1754670
Data columns (total 25 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   patient_custom_master_age  float64
 1   patient_onsetage           float64
 2   patient_onsetageunit       object 
 3   patient_sex                object 
 4   patient_weight             float64
 5   safetyreportid             object 
 6   nichd                      object 
 7   ich_ema                    object 
 8   fda                        object 
 9   lastupdate_date            int64  
 10  mostrecent_receive_date    int64  
 11  receive_date               int64  
 12  congenital_anomali         float64
 13  death                      float64
 14  disabling                  float64
 15  life_threatening           float64
 16  other                      float64
 17  serious                    object 
 18  reporter_company           object 
 19  reporter_country           object 
 20  re

##### Missing Data

In [19]:
missing_data_merge = ped_data_merge.isna().sum().sort_values(ascending=False)
missing_data_merge

congenital_anomali           1718001
disabling                    1654300
reaction_outcome             1653173
life_threatening             1537713
death                        1429485
patient_weight                906664
drug_indication               873247
other                         866181
fda                           699833
ich_ema                       479374
reporter_country              292501
reporter_qualification        288592
reporter_company               99597
patient_sex                    37526
medicinal_product                 34
patient_custom_master_age          0
patient_onsetage                   0
safetyreportid                     0
nichd                              0
patient_onsetageunit               0
receive_date                       0
mostrecent_receive_date            0
lastupdate_date                    0
serious                            0
drug_characterization              0
dtype: int64

#### Age Overview

In [20]:
# check  age unit
ped_data_merge['patient_onsetageunit'].value_counts(ascending=False)

patient_onsetageunit
Year      1498240
Month      157629
Day         78514
Week        13428
Hour         5810
Decade       1050
Name: count, dtype: int64

In [21]:
#Recheck Age after filter
ped_data_merge.loc[(ped_data_merge['patient_onsetageunit'] == 'Month'),['patient_custom_master_age','patient_onsetage','patient_onsetageunit']].sort_values(by='patient_onsetage',ascending=False)

,patient_custom_master_age,patient_onsetage,patient_onsetageunit
1224610,20.916667,251.0,Month
1162528,20.916667,251.0,Month
1162529,20.916667,251.0,Month
1162526,20.916667,251.0,Month
1162527,20.916667,251.0,Month
...,...,...,...
723418,0.083333,1.0,Month
723419,0.083333,1.0,Month
723420,0.083333,1.0,Month
723421,0.083333,1.0,Month


In [22]:
# print(type(ped_data_merge))

In [23]:
# print(ped_data_merge)

In [24]:
# Add new column with 'Year' as the value for all rows
ped_data_merge['patient_custom_master_age_unit'] = 'Year'

# Reorder columns to place the new column after 'patient_custom_master_age'
cols = list(ped_data_merge.columns)
cols.insert(cols.index('patient_custom_master_age') + 1, cols.pop(cols.index('patient_custom_master_age_unit')))
ped_data_merge = ped_data_merge[cols]

In [25]:
ped_data_merge.isna().sum().sort_values(ascending=False)

congenital_anomali                1718001
disabling                         1654300
reaction_outcome                  1653173
life_threatening                  1537713
death                             1429485
patient_weight                     906664
drug_indication                    873247
other                              866181
fda                                699833
ich_ema                            479374
reporter_country                   292501
reporter_qualification             288592
reporter_company                    99597
patient_sex                         37526
medicinal_product                      34
patient_custom_master_age_unit          0
patient_custom_master_age               0
patient_onsetage                        0
safetyreportid                          0
nichd                                   0
patient_onsetageunit                    0
receive_date                            0
mostrecent_receive_date                 0
lastupdate_date                   

In [26]:
ped_data_merge.shape

(1754671, 26)

### Data Cleaning

#### Fill NA and Unknow

In [27]:
#fill NA in columns congenital_anomali,disabling,life_threatening,death,other
cols = ['congenital_anomali','disabling','life_threatening','death','other']
for i in cols:
    print(ped_data_merge[i].unique())

existing_cols = [col for col in cols if col in ped_data_merge.columns]

for col in existing_cols:
    ped_data_merge[col] = ped_data_merge[col].fillna(0).astype(int)
    
# After filling na
print('')
print('After filling NA : \n')
for i in cols:
    print(ped_data_merge[i].unique())

[ 2.  1. nan]
[ 2.  1. nan]
[ 2.  1. nan]
[ 2.  1. nan]
[ 2.  1. nan]

After filling NA : 

[2 1 0]
[2 1 0]
[2 1 0]
[2 1 0]
[2 1 0]


In [28]:
# fill unknown value in columns about patient
categorical_cols_to_fill_unknown = ['patient_sex']
for col in categorical_cols_to_fill_unknown:
    ped_data_merge[col] = ped_data_merge[col].fillna('Unknown')

In [29]:
# fill unknown value in columns about reporter
categorical_cols_to_fill_unknown = ['reporter_company', 'reporter_qualification', 'reporter_country']
for col in categorical_cols_to_fill_unknown:
    ped_data_merge[col] = ped_data_merge[col].fillna('Unknown')

In [30]:
# fill unknown value in columns about reaction
categorical_cols_to_fill_unknown = ['reaction_outcome']
for col in categorical_cols_to_fill_unknown:
    ped_data_merge[col] = ped_data_merge[col].fillna('Unknown')

In [74]:
ped_data_merge.isna().sum().sort_values(ascending=False)

patient_weight                    906664
drug_indication                   873247
fda                               699833
ich_ema                           479374
medicinal_product                     34
patient_custom_master_age              0
patient_sex                            0
patient_onsetageunit                   0
patient_custom_master_age_unit         0
patient_onsetage                       0
safetyreportid                         0
nichd                                  0
lastupdate_date                        0
mostrecent_receive_date                0
death                                  0
disabling                              0
receive_date                           0
congenital_anomali                     0
other                                  0
life_threatening                       0
serious                                0
reporter_company                       0
reporter_qualification                 0
reporter_country                       0
drug_characteriz

In [32]:
ped_data_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1754671 entries, 0 to 1754670
Data columns (total 26 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   patient_custom_master_age       float64
 1   patient_custom_master_age_unit  object 
 2   patient_onsetage                float64
 3   patient_onsetageunit            object 
 4   patient_sex                     object 
 5   patient_weight                  float64
 6   safetyreportid                  object 
 7   nichd                           object 
 8   ich_ema                         object 
 9   fda                             object 
 10  lastupdate_date                 int64  
 11  mostrecent_receive_date         int64  
 12  receive_date                    int64  
 13  congenital_anomali              int64  
 14  death                           int64  
 15  disabling                       int64  
 16  life_threatening                int64  
 17  other                      

#### Datetime format

In [33]:
print(ped_data_merge[['lastupdate_date', 'mostrecent_receive_date', 'receive_date']].head())
date_cols_to_convert = ['lastupdate_date', 'mostrecent_receive_date', 'receive_date']

for col in date_cols_to_convert:
    ped_data_merge[col] = pd.to_datetime(ped_data_merge[col].astype(str), format='%Y%m%d', errors='coerce')

   lastupdate_date  mostrecent_receive_date  receive_date
0         20250409                 20250327      20250327
1         20250409                 20250327      20250327
2         20250409                 20250327      20250327
3         20250409                 20250327      20250327
4         20250409                 20250327      20250327


In [34]:
print(ped_data_merge[['lastupdate_date', 'mostrecent_receive_date', 'receive_date']].head())
print(ped_data_merge.info())

  lastupdate_date mostrecent_receive_date receive_date
0      2025-04-09              2025-03-27   2025-03-27
1      2025-04-09              2025-03-27   2025-03-27
2      2025-04-09              2025-03-27   2025-03-27
3      2025-04-09              2025-03-27   2025-03-27
4      2025-04-09              2025-03-27   2025-03-27
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1754671 entries, 0 to 1754670
Data columns (total 26 columns):
 #   Column                          Dtype         
---  ------                          -----         
 0   patient_custom_master_age       float64       
 1   patient_custom_master_age_unit  object        
 2   patient_onsetage                float64       
 3   patient_onsetageunit            object        
 4   patient_sex                     object        
 5   patient_weight                  float64       
 6   safetyreportid                  object        
 7   nichd                           object        
 8   ich_ema                         

In [35]:
ped_data_merge.shape

(1754671, 26)

#### Droup Data

In [36]:
# Droup medicinal_product because have missing 34 rows
ped_data_merge = ped_data_merge.dropna(subset=['medicinal_product'])

In [37]:
# ลบคอลัมน์ 'patient_weight'
ped_data_merge = ped_data_merge.drop(columns=['patient_weight'])

In [38]:
ped_data_merge =ped_data_merge.dropna(subset=['drug_indication'])

In [39]:
ped_data_merge.isna().sum().sort_values(ascending=False)

fda                               351537
ich_ema                           241452
mostrecent_receive_date               28
patient_custom_master_age_unit         0
patient_custom_master_age              0
patient_sex                            0
patient_onsetageunit                   0
nichd                                  0
patient_onsetage                       0
safetyreportid                         0
lastupdate_date                        0
receive_date                           0
congenital_anomali                     0
death                                  0
disabling                              0
life_threatening                       0
other                                  0
serious                                0
reporter_company                       0
reporter_country                       0
reporter_qualification                 0
drug_characterization                  0
drug_indication                        0
medicinal_product                      0
reaction_outcome

In [40]:
ped_data_merge.shape

(881424, 25)

#### Save Files

In [41]:
ped_data_merge.to_csv('../../data/pediatric_patients_report_drug_reaction.csv.gz',compression='gzip')

In [42]:
data = pd.read_csv(
    '../../data/pediatric_patients_report_drug_reaction.csv.gz',
    compression='gzip',
    low_memory=False,
    index_col=0
)

sampled_data = data.sample(n=1000, random_state=42)

print("ข้อมูลสุ่ม 1,000 แถว:")
print("\nจำนวนแถวและคอลัมน์:", sampled_data.shape)
print("\nคอลัมน์ทั้งหมด:", sampled_data.columns.tolist())
display(sampled_data.head(10))


sampled_data.to_csv('../../data/sampled_pediatric_data.csv', index=False)
sampled_data.to_excel('../../data/sampled_pediatric_data.xlsx', index=False)

ข้อมูลสุ่ม 1,000 แถว:

จำนวนแถวและคอลัมน์: (1000, 25)

คอลัมน์ทั้งหมด: ['patient_custom_master_age', 'patient_custom_master_age_unit', 'patient_onsetage', 'patient_onsetageunit', 'patient_sex', 'safetyreportid', 'nichd', 'ich_ema', 'fda', 'lastupdate_date', 'mostrecent_receive_date', 'receive_date', 'congenital_anomali', 'death', 'disabling', 'life_threatening', 'other', 'serious', 'reporter_company', 'reporter_country', 'reporter_qualification', 'drug_characterization', 'drug_indication', 'medicinal_product', 'reaction_outcome']


,patient_custom_master_age,patient_custom_master_age_unit,patient_onsetage,patient_onsetageunit,patient_sex,safetyreportid,nichd,ich_ema,fda,lastupdate_date,...,life_threatening,other,serious,reporter_company,reporter_country,reporter_qualification,drug_characterization,drug_indication,medicinal_product,reaction_outcome
84619,2.00,Year,2.0,Year,Female,4368498-5,toddler,infants_and_toddlers,children,2005-01-07,...,0,0,The adverse event did not result in any of the...,US-GLAXOSMITHKLINE-A0441452A,Unknown,Consumer or non-health professional,Suspect (the drug was considered by the report...,EAR INFECTION,AUGMENTIN ES-600,Unknown
755635,21.00,Year,21.0,Year,Female,5812637-3,late_adolescence,NaN,NaN,2009-01-09,...,0,1,"The adverse event resulted in death, a life th...",BR-BAYER-200815695LA,BRAZIL,Consumer or non-health professional,Concomitant (the drug was reported as being ta...,MULTIPLE SCLEROSIS,BACLOFEN,Unknown
1129059,13.00,Year,13.0,Year,Male,6845090-1,early_adolescence,adolescents,adolescents,2011-02-19,...,0,1,"The adverse event resulted in death, a life th...",JP-PFIZER INC-2010081337,JAPAN,Physician,Suspect (the drug was considered by the report...,GLOMERULOSCLEROSIS,METHYLPREDNISOLONE SODIUM SUCCINATE,Unknown
1003489,17.00,Year,17.0,Year,Female,6466167-9,early_adolescence,adolescents,NaN,2010-05-25,...,0,0,The adverse event did not result in any of the...,20097640,UNITED STATES,Physician,Suspect (the drug was considered by the report...,MUSCLE SPASTICITY,LIORESAL,Unknown
582367,20.00,Year,20.0,Year,Female,5378133-5,late_adolescence,NaN,NaN,2008-01-15,...,0,1,"The adverse event resulted in death, a life th...",PHEH2007US07534,UNITED STATES,Physician,Suspect (the drug was considered by the report...,IRRITABLE BOWEL SYNDROME,ZELNORM,Unknown
1366099,2.00,Year,2.0,Year,Female,7586805-7,toddler,infants_and_toddlers,children,2011-12-22,...,0,1,"The adverse event resulted in death, a life th...",PHHY2011DK55933,DENMARK,Other health professional,Suspect (the drug was considered by the report...,CORNEAL OPACITY,CHLORAMPHENICOL,Unknown
538130,20.00,Year,20.0,Year,Male,5280765-X,late_adolescence,NaN,NaN,2007-07-07,...,0,1,"The adverse event resulted in death, a life th...",US-PURDUE-USA_2007_0026594,UNITED STATES,Unknown,Suspect (the drug was considered by the report...,DRUG USE FOR UNKNOWN INDICATION,HYDROCODONE,Unknown
898948,1.75,Year,21.0,Month,Female,6183319-6,toddler,infants_and_toddlers,infants,2009-10-09,...,0,0,"The adverse event resulted in death, a life th...",HK-BAXTER-2008BH001568,HONG KONG,Pharmacist,Concomitant (the drug was reported as being ta...,PYREXIA,ACETAMINOPHEN,Unknown
544560,15.00,Year,15.0,Year,Female,5294722-0,early_adolescence,adolescents,adolescents,2007-10-10,...,0,1,"The adverse event resulted in death, a life th...",SPV1-2007-01631,UNITED STATES,Consumer or non-health professional,Suspect (the drug was considered by the report...,ATTENTION DEFICIT/HYPERACTIVITY DISORDER,"ADDERALL XR(AMPHETAMINE ASPARTATE, AMPHETAMINE...",Unknown
783094,17.00,Year,17.0,Year,Male,5887522-1,early_adolescence,adolescents,NaN,2009-01-09,...,0,1,"The adverse event resulted in death, a life th...",PHHY2008AU20749,AUSTRALIA,Consumer or non-health professional,Suspect (the drug was considered by the report...,ABNORMAL BEHAVIOUR,DEXAMFETAMINE,Unknown
